# How to use toolkits with agents

LangChain implements a suite of [Tools](/docs/concepts#tools), as well as collections of tools designed to be used together called [Toolkits](/docs/concepts#toolkits). Tools allow LLMs to interact with the world by taking actions and observing results. Common examples include interacting with structured data in a database, searching the web, and interacting with APIs. See [integrations](/docs/integrations/toolkits/) for more examples.

To create an agent and equip it with tools, we generally need three things: (1) a LLM; (2) the tools; and (3) a prompt. These are easily accessible:

1. LangChain implements integrations with most [LLMs](https://python.langchain.com/v0.2/docs/integrations/chat/).
2. [Tools](/docs/concepts#tools) and [Toolkits](/docs/concepts#toolkits) can be imported from a LangChain library-- typically `langchain-community`. See integrations [here](/docs/integrations/toolkits/).
3. Prompts can be downloaded from the [LangSmith Hub](https://smith.langchain.com/hub), edited, or entirely customized.

Once we have these three, we can incorporate them into an agent architecture. There are two ways of doing this:

1. (Recommended) [LangGraph](https://langchain-ai.github.io/langgraph/) offers simple pre-built agent architectures, and implements a framework for customizing these architectures;
2. (Legacy) The [AgentExecutor](/docs/how_to/agent_executor/) class in `langchain` implements a class of agent architectures that can be used as well.

Below, we demonstrate how to equip agents with a selection of common toolkits. We go through one example in-depth (SQL agents), and provide concise snippets that can be easily copy and pasted for the others.

**Note**: This guide requires these dependencies:

In [ ]:
%pip install --upgrade --quiet langchain langchain_community langchainhub langgraph

## ⚠️ Security note ⚠️

There are inherent risks in giving models discretion to execute real-world actions. Take precautions to mitigate these risks:

- Make sure that permissions associated with the tools are narrowly-scoped (e.g., for database operations or API requests);
- When desired, make use of [human-in-the-loop](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/breakpoints/) workflows.

## Example: SQL agent

### 1. Select LLM

First we select a LLM. Any LLM supporting [tool calling](/docs/how_to/tool_calling) will suffice.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [1]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

### 2. Load tools

Next we load the tools. What we ultimately need is a list of [BaseTool](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.BaseTool.html) objects. These can be assembled from individual tools. Toolkits also provide conveniences for loading sets of tools. Below we use the [SQLDatabaseToolkit](https://api.python.langchain.com/en/latest/agent_toolkits/langchain_community.agent_toolkits.sql.toolkit.SQLDatabaseToolkit.html) from `langchain-community`.

We will use a local SQLite database in this example. Setup instructions for this database are available in the SQL question-answering tutorial [here](https://python.langchain.com/v0.2/docs/tutorials/sql_qa/#setup).

In [2]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

### 3. Create prompt

Here we will pull an instructive system message from the [LangSmith Hub](https://smith.langchain.com/hub).

Importantly, prompts may feature parameters that should be specified. We can inspect these parameters via the `.input_variables` attribute of the prompt template:

In [3]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)
prompt_template.messages[0].pretty_print()

['dialect', 'top_k']
================================ System Message ================================

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DEL

Here, we should specify the relevant SQL dialect, as well as the row limit for queries. Formatting choices for these parameters creates a string system message for our agent.

These parameters represent common variants for a SQL agent prompt, but you should feel free to edit the entirety of the prompt to tune it to your use-case.

In [4]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)

### Create agent

Finally, we create and run our agent. We demonstrate this with LangGraph and AgentExecutor.

#### Langgraph (recommended)

[LangGraph](https://langchain-ai.github.io/langgraph/) implements a framework for building custom and controllable agents. It offers several advantages, such as built-in persistence, enabling advanced human-in-the-loop and memory features.

LangGraph also includes several pre-built agent architectures. Here, we will create a simple ReAct-style agent:

In [5]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

We can then issue a query to the agent and observe the steps it takes in generating an answer. Note that it calls multiple tools that present available tables and their schemas before finally writing a SQL query.

In [6]:
events = agent_executor.stream(
    {"messages": [("user", "Which country's customers spent the most?")]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Which country's customers spent the most?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_N39a7l5QFeY60AQbZ9XzdFmU)
 Call ID: call_N39a7l5QFeY60AQbZ9XzdFmU
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_YdTP2DsBmwxmWJciNzmjKc5H)
 Call ID: call_YdTP2DsBmwxmWJciNzmjKc5H
  Args:
    table_names: Customer, Invoice, InvoiceLine
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL

#### AgentExecutor (legacy)

`langchain` includes an [AgentExecutor](/docs/how_to/agent_executor) class that will also enable LLMs to execute tools and observe their results. `AgentExecutor` implements a class of ReAct-style agent architectures in which observations are recorded in a designated section of the prompt called a "scratchpad". For this reason, it requires a special prompt template.

We can load this template from the LangSmith Hub and format in our system message from before:

In [7]:
prompt_template = hub.pull("langchain-ai/tools-agent")

print(prompt_template.input_variables)

prompt = prompt_template.partial(system_message=system_message)

['agent_scratchpad', 'input', 'system_message']


Finally, we create and execute the agent:

In [8]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent_executor = AgentExecutor(
    agent=create_tool_calling_agent(llm, tools, prompt),
    tools=tools,
    verbose=True,
)

In [9]:
agent_executor.invoke({"input": "Which country's customers spent the most?"})



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer, Invoice, InvoiceLine'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 2170	

{'input': "Which country's customers spent the most?",
 'output': 'Customers from the USA spent the most, with a total amount spent of $523.06.'}

## Example: web search

### Tools

### Prompt

## Example: API agent

### Tools

### Prompt